Module : <b>Blockchain</b><br> 
Travail réalisé par :<b>Mehdi AMOR OUAHMED</b><br>
<br> Niveau : <b>3 DSIA</b>  <br> Année Universitaire : <b>2023-2024</b>
# Interagir avec un smart contrat<br>

Suite à des recherches, j'ai déterminé que la meilleure méthode pour interagir avec un smart contract est l'utilisation de la bibliothèque `web3.py`. <br>
Pour acquérir une compréhension détaillée, j'ai consulté une playlist sur YouTube qui détaille les étapes essentielles à suivre : https://www.youtube.com/playlist?list=PLw-9a9yL-pt3oKQgq_ZYzkMKAkbr8M67P . <br>
En parallèle, j'ai exploré la documentation officielle de la bibliothèque, accessible ici : https://web3py.readthedocs.io/en/stable/quickstart.html .

## I. Se connecter au réseau de testnet BNB
Pour avoir le lien du provider, j'ai consulté : https://chainlist.org/chain/97

In [1]:
from web3 import Web3

In [2]:
test_net_bnb_url = "https://data-seed-prebsc-1-s1.binance.org:8545"
w3 = Web3(Web3.HTTPProvider(test_net_bnb_url))

if w3.is_connected():
    print("Connected to BSC Testnet")
else:
    print("Connection failed")

Connected to BSC Testnet


## II. Importation du contrat
L'abi est dans un fichier qui s'appelle `abi.json` dans le dossier `src`

In [3]:
import json
import os
f = open(os.path.join('src', 'abi.json'))
contract_abi = json.load(f)
contract_address = "0x660aCEc32977F30aEe4A8637FE489dA43be3302e"

contract = w3.eth.contract(address=contract_address, abi=contract_abi)

## III. Intergir avec le contrat
### 1. Les fonctions de lecture:
Ces fonctions ne demande pas de faire une transaction, i.e on n'a pas besoin de compte.

In [4]:
contract.functions.nombre_taches_accomplies().call()

0

In [5]:
contract.functions.voir_progres().call()

[[], []]

### 2. Les fonctions d'écriture :
Ces fonctions demande pas de faire une transaction, i.e on a besoin d'un compte.<br>
Pour utiliser ma clé privée sans la mentionner dans le code j'ai utilisé un fichier `.env`

In [6]:
from dotenv import find_dotenv, load_dotenv
dotenv_path = find_dotenv()
load_dotenv(dotenv_path)

True

In [7]:
wallet = "0x0be5E8Df43124d3B94e88fF0e6C866f1A4891893"
private_key = os.getenv("private_key")

#### Ajouter une tâche

In [8]:
add_task_transaction = contract.functions.ajouter_une_tache('Faire le DM du module Blockchain').build_transaction(
    {"gasPrice": w3.eth.gas_price, 
     "chainId": w3.eth.chain_id, 
     "from": wallet, 
     "nonce": w3.eth.get_transaction_count(wallet)})

In [9]:
# Signer le contrat avec mon compte
signed_transaction = w3.eth.account.sign_transaction(add_task_transaction, private_key=private_key)

In [10]:
transaction_hash = w3.eth.send_raw_transaction(signed_transaction.rawTransaction)
transaction_receipt = w3.eth.wait_for_transaction_receipt(transaction_hash)

In [11]:
# Vérifier que ça marche bien
contract.functions.voir_progres().call()

[['Faire le DM du module Blockchain'], [False]]

Bingo !

#### Mettre une tâche accomplie

In [12]:
add_task_transaction = contract.functions.tache_accomplie('Faire le DM du module Blockchain').build_transaction(
    {"gasPrice": w3.eth.gas_price, 
     "chainId": w3.eth.chain_id, 
     "from": wallet, 
     "nonce": w3.eth.get_transaction_count(wallet)})

signed_transaction = w3.eth.account.sign_transaction(add_task_transaction, private_key=private_key)

transaction_hash = w3.eth.send_raw_transaction(signed_transaction.rawTransaction)

transaction_receipt = w3.eth.wait_for_transaction_receipt(transaction_hash)

In [13]:
# Vérifier que ça marche bien
contract.functions.voir_progres().call()

[['Faire le DM du module Blockchain'], [True]]

In [14]:
contract.functions.pourcentage_accompli().call()

100

#### Reset

In [15]:
add_task_transaction = contract.functions.reset().build_transaction(
    {"gasPrice": w3.eth.gas_price, 
     "chainId": w3.eth.chain_id, 
     "from": wallet, 
     "nonce": w3.eth.get_transaction_count(wallet)})

signed_transaction = w3.eth.account.sign_transaction(add_task_transaction, private_key=private_key)

transaction_hash = w3.eth.send_raw_transaction(signed_transaction.rawTransaction)

transaction_receipt = w3.eth.wait_for_transaction_receipt(transaction_hash)

In [16]:
# Vérifier que ça marche bien
contract.functions.voir_progres().call()

[[], []]